In [ ]:
import sys
import os

os.chdir("..")
os.chdir("./src")
# sys.path.append("./src")

In [ ]:
from IPython import display
from IPython.display import clear_output
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, jit
from time import time

# from helpers import *
from LDMIBSS import *

# np.random.seed(13)
%load_ext autoreload
%autoreload 2

In [ ]:
dim = 6
N = 500000
NumberofSources = dim
NumberofMixtures = 10

signed_dims = np.array([0, 1, 2, 3])
nn_dims = np.array([4, 5])
sparse_dims_list = [np.array([0, 1, 4, 5])]
(A, b), V = generate_practical_polytope(dim, signed_dims, nn_dims, sparse_dims_list)
S = generate_uniform_points_in_polytope(V, N)

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(1, 3, 1)
plt.scatter(S[0, :], S[1, :])
plt.xlabel("$S_{:,1}$", fontsize=25)
plt.ylabel("$S_{:,2}$", fontsize=25)
plt.grid(linewidth=0.2)
plt.subplot(1, 3, 2)
plt.scatter(S[0, :], S[2, :])
plt.xlabel("$S_{:,1}$", fontsize=25)
plt.ylabel("$S_{:,3}$", fontsize=25)
plt.grid(linewidth=0.2)
plt.subplot(1, 3, 3)
plt.scatter(S[1, :], S[2, :])
plt.xlabel("$S_{:,2}$", fontsize=25)
plt.ylabel("$S_{:,3}$", fontsize=25)
plt.grid(linewidth=0.2)

plt.suptitle("Scatter Plot of Source Components", fontsize=30)
# plt.savefig('Pex_source_components.pdf', format='pdf', dpi = 1200)

In [ ]:
# Generate Mxr random mixing from i.i.d N(0,1)
A = np.random.randn(NumberofMixtures, NumberofSources)
X = np.dot(A, S)

SNR = 30  # dB
NoiseAmp = 10 ** (-SNR / 20)  # * np.sqrt(NumberofSources)
NoisePart = NoiseAmp * np.random.randn(X.shape[0], X.shape[1])
X = X + NoisePart
SNRinp = 10 * np.log10(np.sum(np.var(X, axis=1)) / np.sum(np.var(NoisePart, axis=1)))
print("The following is the mixture matrix A")
display_matrix(A)
print("Input SNR is : {}".format(SNRinp))

In [ ]:
subplot_1D_signals(
    S.T[0:100], title="Original Signals", figsize=(15.2, 9), colorcode=None
)
subplot_1D_signals(
    X.T[0:100], title="Mixture Signals", figsize=(15, 18), colorcode=None
)

In [ ]:
lambday = 1 - 1e-2 / 2
lambdae = 1 - 1e-5 / 10
beta = 50
s_dim = S.shape[0]
x_dim = X.shape[0]

# Inverse output covariance
By = 5 * np.eye(s_dim)
# Inverse error covariance
Be = 1 * np.eye(s_dim)

debug_iteration_point = 1000
model = OnlineLDMIBSS(
    s_dim=s_dim,
    x_dim=x_dim,
    muW=1e-3,
    lambday=lambday,
    lambdae=lambdae,
    beta=beta,
    By=By,
    Be=Be,
    neural_OUTPUT_COMP_TOL=1e-6,
    set_ground_truth=True,
    S=S,
    A=A,
)

In [ ]:
model.fit_batch_general_polytope(
    X=X,
    signed_dims=signed_dims,
    nn_dims=nn_dims,
    sparse_dims_list=sparse_dims_list,
    n_epochs=1,
    neural_dynamic_iterations=500,
    plot_in_jupyter=True,
    neural_lr_start=0.1,
    neural_lr_stop=1e-5,
    debug_iteration_point=debug_iteration_point,
    shuffle=True,
)

In [ ]:
a = np.array([-1])

In [ ]:
model.W